---
Lógica Computacional TP1.Exercício1
======================
##### G01 - Bruno Dias da Gião A96544, João Luis da Cruz Pereira A95375

---

## Exercício 1 - Enunciado

1. Pretende-se construir o horário semanal de aulas de uma turma.
    1. Existe um conjunto de salas $S$ classificadas em “grandes” e “pequenas”.
    2. Existe um conjunto  $T_1$  de períodos de tempo de 1 hora e um conjunto $T_2$ de períodos de tempo de duas horas. O total do tempo disponível é 5 horas de manhã e 5 horas às tarde.
    3. Existe um conjunto $D\,$ de disciplinas.  Cada disciplina tem um atributo $d\in\{1,2\}$ que classifica a duração de cada aula, um atributo $a\in\{2,3\}$ que define o número de aulas semanais e um atributo $\,s\in\{0,1\}\,$ que diz se a aula necessita de uma sala grande ou não.
    4. Existe um conjunto $P$ de professores. Cada professor tem associado um conjunto  $h$ das disciplinas que está habilitado a lecionar.
    O horário deve verificar as seguintes restrições
        1. Para cada disciplina todas as aulas decorrem na mesma sala e com o mesmo professor.
        2. Nenhum professor tem duas aulas sem um intervalo de pelo menos uma hora.
        3. O número total de horas lecionadas por cada professor está num intervalo de $\pm 20\%\,$do número médio de horas lecionadas pela totalidade dos professores.
        4. Nenhuma sala pode ser ocupada simultaneamente por mais do que uma aula e nenhum professor pode lecionar simultaneamente mais do que uma aula.
        5. Em cada disciplina, cada aula é lecionada por um professor habilitado para essa disciplina e ocorre numa sala de tamanho apropriado à disciplina.


    Use o package `ortools` para encontrar uma solução que verifique todas as restrições e maximize o número de partes de dia (manhãs ou tardes) que estão livres de qualquer aula.
    
---

## Exercício 1 - Solução

### Exercício 1 - Código

### Exercício 1 - Testes